In [2]:
#----------------------------------------------------------------------------
# Created By: Danny Camenisch (dcamenisch)
# Created Date: 10/03/2022
# version ='1.1'
# ---------------------------------------------------------------------------
""" 
Simple module to convert a xml file containing data in TRACAB format to a Match object
"""
# ---------------------------------------------------------------------------
import xml.etree.ElementTree as et

class Match:
    def __init__(self, filePath):
        match = et.parse(filePath).getroot()[0]

        self.matchID      = int(match.attrib['id'])
        self.matchNr      = int(match.attrib['matchNumber'])
        self.date         = match.attrib['dateMatch']
        self.stadiumID    = int(match[1].attrib['id'])
        self.stadiumName  = match[1].attrib['name']
        self.pitchLength  = int(match[1].attrib['pitchLength'])
        self.pitchWidth   = int(match[1].attrib['pitchWidth'])
        self.phases       = [Phase(phase) for phase in match[2]]
        self.frames       = [Frame(frame) for frame in match[3]]

        self.removeExcessFrames()
        
    def removeExcessFrames(self):
        keep = []
        for frame in self.frames:
            for phase in self.phases:
                if frame.time >= phase.start and frame.time <= phase.end:
                    keep.append(frame)
                    break

        self.frames = keep


class Phase:
    def __init__(self, phase):
        self.start       = phase.attrib['start']
        self.end         = phase.attrib['end']
        self.leftTeamID  = int(phase.attrib['leftTeamID'])
        
class Frame:
    def __init__(self, frame):
        self.time            = frame.attrib['utc']
        self.ballInPlay      = frame.attrib['isBallInPlay']
        self.ballPossession  = frame.attrib['ballPossession']
        self.trackingObjs    = [TrackingObj(obj) for obj in frame[0]]
    
class TrackingObj:
    def __init__(self, obj):
        self.type      = obj.attrib['type']
        self.id        = obj.attrib['id']
        self.x         = int(obj.attrib['x'])
        self.y         = int(obj.attrib['y'])
        self.sampling  = obj.attrib['sampling']


In [3]:
import numpy as np

def get_player_distance_covered(match, id):
    """
    Following @dcamenisch's match parser object, insert a player id
    to get the distance covered in meters.

    Parameters
    ----------
    match: match object parsed

    id: str
        player id in strings
    
    Returns
    ---------
    @return: float
        total distance covered for the player in meters.
    """
    x_c = []
    y_c = []
    num_frames = len(match.frames)
    for f in range(num_frames):
        for p in match.frames[f].trackingObjs:
            if((p.id) == id):
                x_c.append(int(p.x)/100)
                y_c.append(int(p.y)/100)

    x_c_nump = np.asarray(x_c)
    y_c_nump = np.asarray(y_c)
    return np.sum(np.sqrt(np.add(np.square(x_c_nump[1:] - x_c_nump[0:-1]), np.square(y_c_nump[1:] - y_c_nump[0:-1]))))


In [4]:
match = Match("../data/tracab/Group Stage/Austria v North Macedonia/Austria v North Macedonia.xml") 

#problem with file not found error, will ask on Friday - note that Austria v North Macedonia.xml
#does not seem to be imported correctly

In [10]:
#to do: loop through player IDs to get covered distances.-ask Clement or Adrian
#----------------------------------------------------------------------------
# Created By: Danny Camenisch (dcamenisch)
# Created Date: 19/03/2022
# version ='1.0'
# ---------------------------------------------------------------------------
""" 
Simple module to parse the UEFA website to get player id's, names, positions and stats
for a given team.
"""
# ---------------------------------------------------------------------------
import requests
from bs4 import BeautifulSoup

baseURL = "https://www.uefa.com/european-qualifiers/teams/"

class Player:
    def __init__(self, attr):
        self.id             = attr[0]
        self.name           = attr[1]
        self.number         = attr[2]
        self.position       = attr[3]
        self.age            = attr[4]
        self.gamesPlayed    = attr[5]
        self.goals          = attr[6]

def parseSite(teamID):
    teamURL = baseURL + str(teamID) + "/squad/"

    # Get the HTML from the site
    page = requests.get(teamURL)

    # Check for bad status code
    if not page.status_code == 200:
        print("Error: Could not get the page")
        return

    # Parse the HTML
    html = page.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # The HTML has multiple divs with class 'squad--team-wrap' that contain all 
    # the player infos split by postition
    squad = soup.find_all('div', class_='squad--team-wrap')

    # Extract the player infos from the HTML
    players = []

    for wrap in squad[1:5]:
        postition = wrap.h5.text[:-1]

        for player in wrap.find('tbody').find_all('tr'):
            infos = player.find_all('td')

            players.append(Player([
                infos[0].find('a', class_='player-name')["href"].split("/")[-2].split("-")[0], 
                infos[0].find('a', class_='player-name')["title"], 
                infos[0].text.strip().split(" ")[-1].strip("()"), 
                postition, 
                infos[1].text.strip(), 
                infos[2].text.strip().replace("-", "0"), 
                infos[3].text.strip().replace("-", "0")
            ]))

    return players


In [16]:
macedonia = parseSite(59205)
for i in macedonia:
    print (i.name + ": "+ i.id)

Stole Dimitrievski: 250014421
Risto Jankov: 250079552
Martin Bogatinov: 94621
Kristijan Naumovski: 96393
Dejan Iliev: 250143636
Damjan Siskovski: 250022908
Egzon Bejtulai: 250055992
Kire Ristevski: 107972
Visar Musliu: 250022910
Stefan Ristovski: 250004363
Darko Velkovski: 250031588
Gjoko Zajkov: 250031589
Nikola Serafimov: 250120802
Todor Todoroski: 250079561
Stefan Ashkovski: 250004361
Arijan Ademi: 250005231
Eljif Elmas: 250091841
Ezgjan Alioski: 250028268
Nikola Gjorgjev: 250068980
Enis Bardi: 250064634
Ferhan Hasani: 250022169
Darko Churlinov: 250105582
Boban Nikolov: 250043103
Erdal Rakip: 250058142
Ennur Totre: 250083763
David Babunski: 250022918
Valon Ethemi: 250120369
Stefan Spirovski: 107976
Jani Atanasov: 250089433
Tihomir Kostadinov: 250055403
Bojan Miovski: 250113373
Ivan Tričkovski: 73471
Marjan Radeski: 250059633
Aleksandar Trajkovski: 250039878
Goran Pandev: 52148
Daniel Avramovski: 250031583
Adis Jahović: 105089
Dorian Babunski: 250042656
Vlatko Stojanovski: 250116833
